In [1]:
import sys

sys.path.append('E:/UADY/CARLA/CARLA_Latest/WindowsNoEditor/PythonAPI/carla')

import carla
import numpy as np
import re
import cv2

In [2]:
client = carla.Client('localhost', 2000)
world = client.get_world()

In [7]:
client.load_world('Town05')

RuntimeError: time-out of 5000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000

In [3]:
car_tesla = world.get_blueprint_library().filter('*model3*')

# vehicle_all = world.get_blueprint_library().filter('*vehicle*')
# vehicle_tesla = world.try_spawn_actor(car_tesla[0], parking_points[1])

# Expresión regular negativa para excluir múltiples patrones
exclude_regex = re.compile(
    r'.*(ambulance|fusorosa|vespa|kawasaki|crossbike|harley|micro|carlamotors|yamaha|gazelle|diamondback).*')

# Filtrar los vehículos excluyendo los patrones especificados
vehicle_all = [bp for bp in world.get_blueprint_library().filter('*vehicle*') if not exclude_regex.match(bp.id)]


In [4]:
def is_available_parking():
    return np.random.choice([False] + [True] * 2)  #probabily of parking

In [5]:
def generate_parking_points(lines_of_parking_x, lines_of_parking_y):
    parking_points = []
    for x in lines_of_parking_x:
        for y in lines_of_parking_y:
            if is_available_parking():
                parking_points.append(
                    carla.Transform(
                        carla.Location(x=x, y=y, z=1),
                        carla.Rotation(yaw=int(np.random.choice([0, 180])))
                    )
                )
    return parking_points

In [6]:
parking_points = []
lines_of_parking_x = [10, 3.5, -7.0, -13.0, -23, -29]
lines_of_parking_y = [-18.8, -21.6, -24.4, -27.2, -30.0, -32.8, -35.6, -38.4, -41.2, -44.0]

spectator_line = lines_of_parking_x[1]
del lines_of_parking_x[1]
parking_points = generate_parking_points(lines_of_parking_x, lines_of_parking_y)

In [9]:
for point in parking_points:
    ramdom_vehicle = np.random.choice(vehicle_all)
    vehicle = world.try_spawn_actor(ramdom_vehicle, point)
    # print(ramdom_vehicle.id)

In [10]:
CAMERA_POS_Z = 3
CAMERA_POS_X = -5

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640')  # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')
camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x=CAMERA_POS_X))

In [11]:
# for y in lines_of_parking_y:
check_parking_point = carla.Transform(
    carla.Location(x=spectator_line, y=-30.0, z=1),
    carla.Rotation(yaw=int(np.random.choice([0, 180])))
)
vehicle_tesla = world.try_spawn_actor(car_tesla[0], check_parking_point)


# vehicle_tesla.destroy()

In [12]:
CAMERA_POS_Z = 3
CAMERA_POS_X = -5

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '360')
camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle_tesla)


def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))


image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h, image_w, 4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image, camera_data))

In [13]:
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])

In [8]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()


In [107]:
vehicle_pos = vehicle_tesla.get_transform()
print(vehicle_pos)

NameError: name 'vehicle_tesla' is not defined

In [19]:
vehicle.set_autopilot(True)